In [30]:
import $ivy.`com.typesafe.akka::akka-stream:2.6.4`
import $ivy.`com.typesafe.akka::akka-http:10.1.11`
// repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-http_2.13/10.1.11/akka-http_2.13-10.1.11.pom
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-http-core_2.13/10.1.11/akka-http-core_2.13-10.1.11.pom
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-parsing_2.13/10.1.11/akka-parsing_2.13-10.1.11.pom
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-http_2.13/10.1.11/akka-http_2.13-10.1.11-sources.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-parsing_2.13/10.1.11/akka-parsing_2.13-10.1.11-sources.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-http-core_2.13/10.1.11/akka-http-core_2.13-10.1.11-sources.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-http_2.13/10.1.11/akka-http_2.13-10.1.11.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akka/akka-parsing_2.13/10.1.11/akka-parsing_2.13-10.1.11.jar
Downloaded https://repo1.maven.org/maven2/com/typesafe/akk

import $ivy.$                                     

import $ivy.$                                     
// repl.pprinter() = repl.pprinter().copy(defaultHeight = 5 )

In [0]:
import akka.stream._
import akka.stream.scaladsl._

cmd0.sc:1: not found: value akka
import akka.stream._
       ^cmd0.sc:2: not found: value akka
import akka.stream.scaladsl._
       ^Compilation Failed

: 

In [ ]:
import akka.{ Done, NotUsed }
import akka.actor.ActorSystem
import akka.util.ByteString
import scala.concurrent._
import scala.concurrent.duration._
import java.nio.file.Paths

In [ ]:
import com.typesafe.config._

In [ ]:
val customConf = ConfigFactory.parseString("""
  blocking-dispatcher {
    executor = "thread-pool-executor"
    thread-pool-executor {
      core-pool-size-min    = 10
      core-pool-size-max    = 10
    }
  }

""")

In [ ]:
implicit val system: ActorSystem = ActorSystem("QuickStart", ConfigFactory.load(customConf))

In [ ]:
implicitly[Materializer]

In [ ]:
implicit val ec = system.dispatcher

In [ ]:
val s1: Source[Int, NotUsed] = Source(List(1,2,3,4))

In [ ]:
val source: Source[Int, NotUsed] = Source(1 to 10)

In [ ]:
val f: Future[Done] = source.runForeach(i => println(i))

In [ ]:
val f: Future[Done] = source.runWith(Sink.foreach(println))

In [ ]:
f.onComplete(_ => println("done"))

In [ ]:
val factorials = source.scan(BigInt(1))((acc, next) => acc * next)

val result: Future[IOResult] =
  factorials.map(num => ByteString(s"$num\n")).runWith(FileIO.toPath(Paths.get("factorials.txt")))

In [ ]:
FileIO.toPath(Paths.get("factorials.txt"))

In [ ]:
Sink.foreach(println)

In [ ]:
def lineSink(filename: String) = // : Sink[String, Future[IOResult]] =
  Flow[String].map(s => ByteString(s + "\n")).toMat(FileIO.toPath(Paths.get(filename)))(Keep.both)

In [ ]:
def lineSinkNU(filename: String): Sink[String, NotUsed] =
  Flow[String]
    .map(s => ByteString(s + "\n"))
    .toMat(FileIO.toPath(Paths.get(filename)))((_, _) => NotUsed)

In [ ]:
source.map(_.toString).runWith(lineSink("factorials2.txt"))

In [ ]:
res16.onComplete(r => println(r))

In [ ]:
source.buffer(3, OverflowStrategy.dropHead).runWith(Sink.foreach(println))

In [ ]:
source.via(Flow[Int].map(s => ByteString(s + "\n")))
    .runWith(FileIO.toPath(Paths.get("f")))

In [ ]:
source.via(Flow[Int].map(s => s + ""))
    .toMat(Sink.fold[String, String]("")(_ + _))(Keep.right)
    .run

In [ ]:
source.runWith(Sink.fold(0)(_+_))

# Buffers and working with rate

In [ ]:
Source(1 to 3)
  .map { i =>
    println(s"A: $i"); i
  }
  .async
  .map { i =>
    println(s"B: $i"); i
  }
  .async
  .map { i =>
    println(s"C: $i"); i
  }
  .async
  .runWith(Sink.ignore)

In [ ]:
Source(1 to 3)
  .map { i =>
    println(s"A: $i"); i
  }
  .map { i =>
    println(s"B: $i"); i
  }
  .map { i =>
    println(s"C: $i"); i
  }
  .runWith(Sink.ignore)

# Futures interop

In [ ]:
import java.util.concurrent.atomic.AtomicInteger

In [ ]:
class SometimesSlowService(implicit ec: ExecutionContext) {

  private val runningCount = new AtomicInteger

  def convert(s: String): Future[String] = {
    println(s"running: $s (${runningCount.incrementAndGet()})")
    Future {
      if (s.nonEmpty && s.head.isLower)
        Thread.sleep(500)
      else
        Thread.sleep(20)
      println(s"completed: $s (${runningCount.decrementAndGet()})")
      s.toUpperCase
    }
 }
}

In [ ]:
val service = new SometimesSlowService()(
    system.dispatchers.lookup("blocking-dispatcher")
)

In [ ]:
Source(List("a", "B", "C", "D", "e", "F", "g", "H", "i", "J"))
  .map(elem => { println(s"before: $elem"); elem })
  .mapAsync(6)(service.convert)
  .to(Sink.foreach(elem => println(s"after: $elem")))
  .withAttributes(Attributes.inputBuffer(initial = 4, max = 4))
  .run()

# Operators

In [9]:
import akka.event.Logging

implicit class SourceOps[A, M](S: Source[A, M]){
    def logAll(l: String): Source[A, M] = 
        S.log(l).withAttributes(Attributes.logLevels(
        onElement = Logging.WarningLevel, 
        onFinish = Logging.WarningLevel, 
        onFailure = Logging.DebugLevel))
}

import akka.event.Logging


defined class SourceOps

In [11]:

Source(List(1,2,3,4)).logAll("source")
    .takeWhile(_ <= 2).logAll("take while")
    .concat(Source(List(5,6,7))).logAll("concat")
    .runWith(Sink.ignore)

[WARN] [03/28/2020 18:18:48.458] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/system/Materializers/StreamSupervisor-0)] [source] Element: 1
[WARN] [03/28/2020 18:18:48.458] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/system/Materializers/StreamSupervisor-0)] [take while] Element: 1
[WARN] [03/28/2020 18:18:48.458] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/system/Materializers/StreamSupervisor-0)] [concat] Element: 1
[WARN] [03/28/2020 18:18:48.459] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/system/Materializers/StreamSupervisor-0)] [source] Element: 2
[WARN] [03/28/2020 18:18:48.459] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/system/Materializers/StreamSupervisor-0)] [take while] Element: 2
[WARN] [03/28/2020 18:18:48.459] [QuickStart-akka.actor.default-dispatcher-9] [akka.stream.Log(akka://QuickStart/syst

res10: Future[Done] = Success(Done)

In [19]:
Flow[Int].map(_ + 1)
         .join[NotUsed](Flow[Int].map(_ + 1))
    .run

res18: NotUsed = NotUsed

In [20]:
Tcp().outgoingConnection("127.0.0.1", 8888)

res19: Flow[ByteString, ByteString, Future[Tcp.OutgoingConnection]] = Flow(FlowShape(OutgoingTCP.in(675478360),GraphStages$Detacher.out(1916872162)))

In [23]:
Source(List(1,2,3))
    .via(Flow[Int].map(_+1)
            .via(Flow[Int].map(_ + 1)))
    .runForeach(println)

3
4
5


res22: Future[Done] = Success(Done)

In [25]:
Source(List(1,2,3))
    .via(Flow[Int].map(_+1)
            .map(_ + 1))
    .runForeach(println)

3
4
5


res24: Future[Done] = Success(Done)

In [35]:
val queue = Source.queue[Int](5, OverflowStrategy.dropNew)
    .map(_ + 1)
    .to(Sink.foreach(println))
    .run

queue: SourceQueueWithComplete[Int] = akka.stream.impl.QueueSource$$anon$1@6f40292f

In [44]:
for(i <- List(4,5,6,7)) queue.offer(3)

In [45]:
queue.watchCompletion.onComplete{ case Success(t) => println("succeed"); 
                                  case Failure(f) => println("failed")
                                  }

succeed


In [46]:
queue.complete